<a href="https://colab.research.google.com/github/photochemistry/andersan-api/blob/main/%E5%8D%B3%E6%99%82%E5%9C%B0%E5%9B%B3%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colabでも動くように修正。でも無理がある。

In [1]:
try:
    # On Google Colab
    from google.colab import drive

    # drive.mount("/content/drive")

    # 大気監視API
    ! pip install git+https://github.com/vitroid/airpollutionwatch.git
    # andersan APIのコア
    ! pip uninstall -y andersan-api
    ! pip install git+https://github.com/photochemistry/andersan-api.git

except:
    pass

In [2]:
from andersan_api.backend import observes
from logging import getLogger, basicConfig, INFO, DEBUG

basicConfig(level=DEBUG)
# 特定の日時の各地理院メッシュ点での測定値をとってくる例
isodate = "2024-09-09T11:00+09:00"
zoom = 12
table = observes("kanagawa", isodate, zoom)
table

2024-09-12 16:26:54.160438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-12 16:26:54.177313: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-12 16:26:54.182472: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-12 16:26:54.195416: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-12 16:26:55.030894: W tensorflow/compiler/tf2

,lon,lat,X,Y,Z,NMHC,OX,NOX,TEMP,WX,WY
timestamp,,,,,,,,,,,
2024-09-09 11:00:00+09:00,138.867188,35.675147,3628,1613,12,6.850441,40.782220,2.831086,292.341582,16.861525,2.202772
2024-09-09 11:00:00+09:00,138.955078,35.675147,3629,1613,12,7.807130,41.077358,2.965944,281.410148,26.490235,7.849418
2024-09-09 11:00:00+09:00,139.042969,35.675147,3630,1613,12,8.763819,42.080633,3.289589,284.524827,10.833870,19.226931
2024-09-09 11:00:00+09:00,139.130859,35.675147,3631,1613,12,11.608483,46.218228,4.661485,293.385706,18.575512,3.601383
2024-09-09 11:00:00+09:00,139.218750,35.675147,3632,1613,12,12.246749,44.401408,5.197183,299.111705,16.636200,11.182838
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-09 11:00:00+09:00,139.394531,35.245619,3634,1619,12,7.731771,23.220183,4.662016,301.238038,-6.395265,-15.694180
2024-09-09 11:00:00+09:00,139.482422,35.245619,3635,1619,12,6.724908,18.180494,5.723754,305.619137,-13.446640,-14.538603
2024-09-09 11:00:00+09:00,139.570312,35.245619,3636,1619,12,5.914255,14.836001,7.580453,309.120219,-21.207295,-16.409881


指定時刻のデータをとってきて地図にする流れ。

In [17]:
# 地図関連の関数
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
import folium
import numpy as np
from andersan_api.backend import prefecture_ranges


def colorize(array, cmap="jet", ra=[0,100]):
    # 値を色に変換
    normed_data = (array - ra[0]) / (ra[1] - ra[0])
    cm = plt.cm.get_cmap(cmap)
    return cm(normed_data)


def map_layer(pref_name, values, name, ra=[0,100]):
    pref_range = prefecture_ranges[pref_name]

    # 色に変換する。NaNは透明になるらしい
    image = colorize(values, ra=ra)

    return folium.raster_layers.ImageOverlay(
        image=image,
        bounds=[
            [np.min(pref_range[:, 1]), np.min(pref_range[:, 0])],  # lat,lon
            [np.max(pref_range[:, 1]), np.max(pref_range[:, 0])],  # lat,lon
        ],
        opacity=0.5,
        name=name,
    )


In [4]:
from folium import Map, LayerControl  # , Choropleth, FeatureGroup
from andersan_api.backend import prefecture_ranges
import numpy as np
from andersan import tile

# 日時の指定
isodate = "2024-09-09T00:00+09:00"
zoom = 12
table = observes("kanagawa", isodate, zoom)
# shape = len(table.Y.unique()), len(table.X.unique())
pref_range = prefecture_ranges["kanagawa"]
tiles, shape = tile.tiles(zoom, pref_range)
# mapに重ねるレイヤ
layers = []

# 測定値
for item in ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]:
    values = table[item].to_numpy()
    layers.append(map_layer("kanagawa", values.reshape(shape), item))

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# GroupedLayerControl({"NOX": layers[0]}, ["NOX"]).add_to(m)

# 表示
m

DEBUG:root:["kanagawa", "2024-09-09T00:00:00+09:00", 12, ["use_amedas", true], ["items", ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]]]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
/tmp/ipykernel_1821286/318765738.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cm = plt.cm.get_cmap(cmap)


6 項目を空間方向で補間し、欠測格子点がなくなった。これを毎日 24 時間前まで入手する。


次は、各格子点の予測値を OpenMeteo から入手して、同じように表示する。

G0L は OM から何を読んでいるか。(datatype1)

```
            "temperature_2m",
            "cloud_cover",
            "pressure_msl",
            "shortwave_radiation",
```

と`weathercode`

OpenMeteo の API では、指定された緯度経度とはすこしずれた場所の情報が送られてきているように思えるがそんなことはないか?


In [5]:
import numpy as np
from logging import getLogger, basicConfig, INFO, DEBUG
from andersan_api.backend import openmeteo_tiles, prefecture_ranges, OPENMETEO_ITEMS
import datetime
from folium import Map, LayerControl  # , Choropleth, FeatureGroup
from andersan_api.backend import prefecture_ranges

basicConfig(level=INFO)

zoom = 12  # 14にしてもOpenMeteoのほうがそこまで解像度がない。
isodate = "2024-08-31T00:00+09:00"
all_forecast_dataframe = openmeteo_tiles("kanagawa", isodate, zoom)
# shape = len(all_forecast_dataframe.Y.unique()), len(all_forecast_dataframe.X.unique())
dt = datetime.datetime.fromisoformat(isodate)
pref_range = prefecture_ranges["kanagawa"]
tiles, shape = tile.tiles(zoom, pref_range)

# レイヤごとに作画する
layers = []
for item in OPENMETEO_ITEMS:
    values = all_forecast_dataframe[all_forecast_dataframe.date == dt][
        [item]
    ].to_numpy()[:, 0]

    # 新しいFoliumレイヤを定義する
    layers.append(map_layer("kanagawa", values.reshape(shape), item))

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# 表示
m

DEBUG:root:["kanagawa", "2024-08-31", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'openmeteo.sqlite'
DEBUG:sqlitedict:closing SqliteDict(openmeteo.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
/tmp/ipykernel_1821286/318765738.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cm = plt.cm.get_cmap(cmap)
/tmp/ipykernel_1821286/318765738.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cm = plt.cm.get_cmap(cmap)
/tmp/ipykernel_1821286/318765738.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colorma

----
ここから先はColabでは動かない。

本番テストに移行する前に: 標準化の方法を統一する。各測定量の標準化方法を固定し、データの多寡によらず同じ標準化が行われるようにする。


In [7]:
import pandas as pd

# import matplotlib.pyplot as plt
# import folium
import numpy as np

# # import requests_cache
# # from retry_requests import retry
from logging import getLogger, basicConfig, INFO, DEBUG
from andersan_api.backend import (
    observes,
    openmeteo_tiles,
    prefecture_ranges,
    OPENMETEO_ITEMS,
    X_instant,
)
import datetime

# from folium import Map, LayerControl  # , Choropleth, FeatureGroup

basicConfig(level=DEBUG)
# 日時の指定
isodate = "2024-09-09T06:00+09:00"
zoom = 12

logger = getLogger()

X = X_instant("kanagawa", isodate, zoom)
X

DEBUG:root:["kanagawa", "2024-09-08T07:00:00+09:00", 12, ["use_amedas", true], ["items", ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]]]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-08T08:00:00+09:00", 12, ["use_amedas", true], ["items", ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]]]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-08T09:00:00+09:00", 12, ["use_amedas", true], ["items", ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]]]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["

{'Input_lookbacks': array([[[-0.95815214, -0.75146911, -0.97589386,  1.10755922,
          -0.23088706,  0.06893295],
         [-0.96570431, -0.46727833, -0.95606382,  1.3461136 ,
          -0.0219229 ,  0.0382355 ],
         [-1.21765907,  0.2969339 , -1.02669516,  1.55919022,
          -0.10447091, -0.15285581],
         ...,
         [-1.26123298, -1.16505758, -0.93970133,  0.96599197,
          -0.39440661, -0.04463037],
         [-1.14426104, -1.10465122, -1.01033267,  0.93645593,
          -0.03964883,  0.18316283],
         [-1.4066745 , -1.12413282, -0.93154003,  0.96547937,
          -0.22731555,  0.23768696]],
 
        [[-1.04402184, -0.86417755, -0.9574227 ,  1.04085874,
          -0.10240691,  0.30918293],
         [-1.05800637, -0.3730559 , -1.02064717,  1.33879395,
           0.22376718,  0.156232  ],
         [-1.35112903,  0.32893015, -1.06616764,  1.57554203,
           0.37031796,  0.33349968],
         ...,
         [-1.37573057, -1.0918241 , -0.94653845,  0.8389556

In [8]:
from datetime import timedelta

# 実測の未来値
forecasts_table = pd.DataFrame()
timeorigin = datetime.datetime.fromisoformat(isodate)
for delta in range(1, 9):
    dt = timeorigin + timedelta(hours=delta)
    table = observes("kanagawa", dt.isoformat(), zoom)
    forecasts_table = pd.concat([forecasts_table, table], axis=0)

forecasts_table
# 14 sec

DEBUG:root:["kanagawa", "2024-09-09T07:00:00+09:00", 12, ["use_amedas", true], ["items", ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]]]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-09T08:00:00+09:00", 12, ["use_amedas", true], ["items", ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]]]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-09T09:00:00+09:00", 12, ["use_amedas", true], ["items", ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]]]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["

,lon,lat,X,Y,Z,NMHC,OX,NOX,TEMP,WX,WY
timestamp,,,,,,,,,,,
2024-09-09 07:00:00+09:00,138.867188,35.675147,3628,1613,12,4.832337,6.271680,4.270857,240.025101,-3.882377,-7.931174
2024-09-09 07:00:00+09:00,138.955078,35.675147,3629,1613,12,5.629473,4.725795,4.949012,234.790831,-6.721175,-13.319992
2024-09-09 07:00:00+09:00,139.042969,35.675147,3630,1613,12,6.426610,3.337650,5.761380,232.278066,-0.234066,-7.385486
2024-09-09 07:00:00+09:00,139.130859,35.675147,3631,1613,12,10.878466,4.308121,7.213509,240.698201,-2.297392,-4.337350
2024-09-09 07:00:00+09:00,139.218750,35.675147,3632,1613,12,11.402705,10.026616,7.673313,253.731341,-3.985066,-3.065131
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-09 14:00:00+09:00,139.394531,35.245619,3634,1619,12,8.000000,16.743977,3.831008,317.668491,-7.520633,-17.743749
2024-09-09 14:00:00+09:00,139.482422,35.245619,3635,1619,12,8.000000,15.854813,3.827067,317.799841,-3.094849,-22.329110
2024-09-09 14:00:00+09:00,139.570312,35.245619,3636,1619,12,8.000000,18.933213,5.174057,315.365941,-4.658945,-17.890371


In [9]:
from andersan import tile

# そのうちOXのみ
# 地理院メッシュの間隔
pref_range = np.array(prefecture_ranges["kanagawa"])  # lon,lat
tiles, shape = tile.tiles(zoom, pref_range)

y = np.zeros([len(tiles), 8])
for j, (tileX, tileY) in enumerate(tiles):
    y[j, :] = forecasts_table[
        (forecasts_table.X == tileX) & (forecasts_table.Y == tileY)
    ].OX.to_numpy()

y
# 0.1 sec

array([[ 6.27168036, 10.72955488, 22.35593132, 33.8980568 , 40.78221983,
        47.31642795, 50.74015469, 58.16182125],
       [ 4.72579529,  8.93937907, 21.29862774, 33.08504396, 41.07735842,
        48.34733712, 55.92497692, 64.67753059],
       [ 3.3376496 ,  7.41537348, 20.49309736, 32.49309736, 42.08063339,
        49.37824629, 61.55992547, 71.50350779],
       [ 4.30812134,  8.75068425, 21.20809641, 33.09863522, 46.21822753,
        56.5065242 , 67.07628841, 76.08572467],
       [10.02661575, 14.02661575, 21.59846991, 32.80320881, 44.40140831,
        55.91592352, 62.47600802, 67.01383956],
       [17.20043826, 19.87765062, 24.78054807, 34.47038385, 44.49563069,
        55.54847273, 65.37328131, 69.87126036],
       [15.59080338, 18.79540169, 24.83888951, 36.04945988, 47.56523173,
        55.86446116, 65.19355088, 71.10657524],
       [13.04585329, 15.10890155, 24.        , 35.57451152, 37.7965843 ,
        47.63182819, 58.74763549, 62.88829089],
       [11.40179031, 12.06872745

In [10]:
import keras

basename = "andersan0_1.py"
model = keras.models.load_model(f"../andersan-train/{basename}.best.keras")
# model.summary()

pred = model.predict(X)
pred**0.5
# 6.7 sec

2024-09-12 16:27:54.664149: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step


array([[11.773608 , 17.094656 , 23.29276  , 29.273802 , 34.44734  ,
        38.500553 , 41.25211  , 42.660652 ],
       [12.285193 , 18.752516 , 25.920624 , 32.78527  , 38.7044   ,
        43.34319  , 46.431973 , 47.888535 ],
       [14.845335 , 21.99556  , 29.524529 , 36.562675 , 42.52973  ,
        47.135994 , 50.143833 , 51.48206  ],
       [17.963282 , 25.127403 , 32.23575  , 38.70432  , 44.046196 ,
        47.975452 , 50.258793 , 50.857746 ],
       [20.061325 , 27.022106 , 33.825634 , 39.97694  , 45.019775 ,
        48.6666   , 50.68907  , 51.061985 ],
       [21.957462 , 31.24326  , 39.922016 , 47.632874 , 53.890293 ,
        58.35955  , 60.75802  , 61.070007 ],
       [18.746277 , 29.248732 , 38.640427 , 46.59832  , 52.415768 ,
        55.551517 , 55.82639  , 53.6373   ],
       [ 8.983047 , 16.064333 , 23.482046 , 30.490261 , 36.59001  ,
        41.474834 , 44.892376 , 46.717274 ],
       [10.046027 , 18.015284 , 26.096228 , 33.7281   , 40.352474 ,
        45.59107  , 49.08032

In [16]:
import folium
from folium import Map, LayerControl, Choropleth, FeatureGroup
import matplotlib.pyplot as plt


def colorize(array, cmap="viridis"):
    # 値を色に変換
    normed_data = array / 120
    cm = plt.cm.get_cmap(cmap)
    return cm(normed_data)


layers = []
foresee = 3
pref_range = prefecture_ranges["kanagawa"]

maps = {"observes": y[:, foresee - 1], "predicts": (pred**0.5)[:, foresee - 1]}
for name, values in maps.items():
    # 新しいFoliumレイヤを定義する
    layers.append(
        map_layer("kanagawa", values.reshape(shape), f"{name} +{foresee} hrs")
    )

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# 表示
m

TypeError: colorize() got an unexpected keyword argument 'ra'

だいぶ簡潔にできた。